# Fedbiomed Researcher base example

Use for developing (autoreloads changes made across packages)

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default) to add MNIST to the node
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

## Define an experiment model and parameters"

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [4]:
from fedbiomed.researcher.environ import environ
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=environ['TMP_DIR']+'/')
model_file = tmp_dir_model.name + '/class_export_mnist.py'

2022-01-21 14:51:56,373 fedbiomed INFO - Component environment:
2022-01-21 14:51:56,374 fedbiomed INFO - - type = ComponentType.RESEARCHER


Note : write **only** the code to export in the following cell

In [5]:
%%writefile "$model_file"

import torch
import torch.nn as nn
from fedbiomed.common.torchnn import TorchTrainingPlan
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self):
        super(MyTrainingPlan, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms",
               "from torch.utils.data import DataLoader"]
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        data_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
        return data_loader
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


Writing /home/scansiz/Desktop/Inria/development/fedbiomed/var/tmp/tmpvfkocdm9/class_export_mnist.py


## Declaring an Experiment by Providing all the Arguments


In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

exp = Experiment(tags=tags,
                 #nodes=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='MyTrainingPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage,
                 node_selection_strategy=None)

In [ ]:
exp.run()

In [ ]:
print('Number of rounds that has ben run    : ' , exp.round_current())
print('Round number for starting next round : ' , exp.round_current() + 1)
print('Round Index                          : ' , list(range(exp.round_current())))

## Declaring an Experiment Step by Step 
### Building Empty Experiment

In [6]:
from fedbiomed.researcher.experiment import Experiment
exp = Experiment()

2022-01-21 14:52:03,230 fedbiomed INFO - Messaging researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f2d19bc8f10>


### Setting Tags 

Tags should list strings that contains tags or a string with single tag. 

---
<div class="note">
    <p>If provided tags is not in correct type `.set_tags` will raise <code>TypeError</code></p>
</div>

In [8]:
tags = ["#MNIST", "#dataset"]
exp.set_tags(tags = tags)

### Setting Model Path and Model Model Class

In [9]:
exp.set_model_path(model_path = model_file)
exp.set_model_class(model_class = 'MyTrainingPlan')

### Setting Model Arguments and Training Arguments

In [10]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100
}

exp.set_model_args(model_args = model_args)
exp.set_training_args(training_args = training_args)

### Setting Training Data

The method `set_trainig_data` gets there arguments: 

- `tags` : List of tags as string for the search request. If it is not provided. The method will try to use `tags` attribute of the object. 
- `nodes`: List of node ids that a search request will be sent. If this argument is not provided search request will be sent to all active nodes.  
- `training_data`: A dictionary or `FederatedDataset` object. If `training_data` provided search request with `tags` and `nodes` will be ignored.

In [11]:
exp.set_training_data()


2022-01-21 14:52:10,782 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-01-21 14:52:10,786 fedbiomed INFO - log from: node_f290cd48-a70a-4e55-9262-81f802f9c95c / DEBUG - Message received: {'researcher_id': 'researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-01-21 14:52:20,795 fedbiomed INFO - Node selected for training -> node_f290cd48-a70a-4e55-9262-81f802f9c95c


### Setting Job 

Setting job will prepare all neccessary assets to be able to run a round. Therefore, `Job` should be set before running the experiment.  

To be able to set `Job`, you should be already set the arguments: `model_path`, `model_class`, `training_data`. Otherwiser `set_job()` will reaise an Exception. 

In [12]:
exp.set_job()

2022-01-21 14:52:22,348 fedbiomed DEBUG - torchnn saved model filename: /home/scansiz/Desktop/Inria/development/fedbiomed/var/experiments/Experiment_0037/my_model_c589df46-1f7d-46c3-b3ad-abcd5e1d2c66.py


True

In [13]:
exp.set_node_selection_strategy()

Parameters of The Experiment

In [16]:
print('Rounds              :', exp.rounds())
print('Tags                :', exp.tags())
print('Model Path          :', exp.model_path())
print('Model Class         :', exp.model_class())
print('Model Arguments     :', exp.model_args())
print('Training Arguments  :', exp.training_args())
print('Job                 :', exp.job())
print('Training Data       :', exp.training_data())
print('Job                 :', exp.job())
print('Nodes               :', exp.nodes()) # Returns selected nodes after search request
print('Aggregator          :', exp.aggregator())
print('N.S. Stragety       :', exp.node_selection_strategy())
print('Breakpoint State    :', exp.breakpoint())
print('Exp  folder         :', exp.experimentation_folder())
print('Exp  path           :', exp.experimentation_path())



Rounds              : 1
Tags                : ['#MNIST', '#dataset']
Model Path          : /home/scansiz/Desktop/Inria/development/fedbiomed/var/tmp/tmpvfkocdm9/class_export_mnist.py
Model Class         : MyTrainingPlan
Model Arguments     : {}
Training Arguments  : {'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}
Job                 : <fedbiomed.researcher.job.Job object at 0x7f2d19bcf8b0>
Training Data       : <fedbiomed.researcher.datasets.FederatedDataSet object at 0x7f2d19bcfaf0>
Job                 : <fedbiomed.researcher.job.Job object at 0x7f2d19bcf8b0>
Nodes               : None
Aggregator          : <fedbiomed.researcher.aggregators.fedavg.FedAverage object at 0x7f2dd818fa60>
N.S. Stragety       : <fedbiomed.researcher.strategies.default_strategy.DefaultStrategy object at 0x7f2d19bcfc70>
Breakpoint State    : False
Exp  folder         : Experiment_0037
Exp  path           : /home/scansiz/Desktop/Inria/development/fedbiomed/var/experiments/Ex

In [15]:
exp.info()

Arguments           Values
------------------  -------------------------------------------------------------------------------------------------------------------
Rounds              1
Tags                ['#MNIST', '#dataset']
Model Path          /home/scansiz/Desktop/Inria/development/fedbiomed/var/tmp/tmpvfkocdm9/class_export_mnist.py
Model Class         MyTrainingPlan
Model Arguments     {}
Training Arguments  {'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}
Nodes
Aggregator          <fedbiomed.researcher.aggregators.fedavg.FedAverage object at 0x7f2dd818fa60>
N.S. Strategy       <fedbiomed.researcher.strategies.default_strategy.DefaultStrategy object at 0x7f2d19bcfc70>
Training Data       <fedbiomed.researcher.datasets.FederatedDataSet object at 0x7f2d19bcfaf0>
Job                 <fedbiomed.researcher.job.Job object at 0x7f2d19bcf8b0>
Breakpoint State    <bound method Experiment._save_breakpoint of <fedbiomed.researcher.experiment.Experiment obj

In [21]:
exp.run_once()

2022-01-21 14:53:33,354 fedbiomed INFO - Round limit reached. Nothing to do


False

In [ ]:
print('Number of rounds initial             : ' , exp.rounds())
print('Number of rounds that has ben run    : ' , exp.round_current())
print('Round number for starting next round : ' , exp.round_current() + 1)
print('Round Indexes                        : ' , list(range(exp.round_current())))

In [ ]:
exp.run_once()

Check current round, deaclare the the round that will be run. 

In [ ]:
print('Number of rounds initial             : ' , exp.rounds())
print('Number of rounds that has ben run    : ' , exp.round_current())
print('Round number for starting next round : ' , exp.round_current() + 1)
print('Round Indexes                        : ' , list(range(exp.round_current())))

Running multiple rounds:

In [23]:
exp.run(rounds=3)

2022-01-21 14:53:54,317 fedbiomed INFO - Sampled nodes in round 1 ['node_f290cd48-a70a-4e55-9262-81f802f9c95c']
2022-01-21 14:53:54,318 fedbiomed INFO - Send message to node node_f290cd48-a70a-4e55-9262-81f802f9c95c - {'researcher_id': 'researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab', 'job_id': '19a32dda-52f8-4d29-a56d-c09de2f12c03', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/21/my_model_c589df46-1f7d-46c3-b3ad-abcd5e1d2c66.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/21/aggregated_params_2b286305-2f16-409c-ab03-a4154d7340d0.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_f290cd48-a70a-4e55-9262-81f802f9c95c': ['dataset_8d7d5b72-228e-4340-8451-169c2470dde6']}}
2022-01-21 14:53:54,318 fedbiomed DEBUG - researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab
2022-01-21 14:53:54,347 fedbiomed INFO - log from: n

2022-01-21 14:54:10,389 fedbiomed INFO - log from: node_f290cd48-a70a-4e55-9262-81f802f9c95c / INFO - Train Epoch: 1 [480/60000 (1%)]	Loss: 0.241935
2022-01-21 14:54:11,076 fedbiomed INFO - log from: node_f290cd48-a70a-4e55-9262-81f802f9c95c / INFO - Train Epoch: 1 [960/60000 (2%)]	Loss: 0.097044
2022-01-21 14:54:11,582 fedbiomed INFO - log from: node_f290cd48-a70a-4e55-9262-81f802f9c95c / INFO - Train Epoch: 1 [1440/60000 (2%)]	Loss: 0.286355
2022-01-21 14:54:12,047 fedbiomed INFO - log from: node_f290cd48-a70a-4e55-9262-81f802f9c95c / INFO - Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.065027
2022-01-21 14:54:12,512 fedbiomed INFO - log from: node_f290cd48-a70a-4e55-9262-81f802f9c95c / INFO - Train Epoch: 1 [2400/60000 (4%)]	Loss: 0.244742
2022-01-21 14:54:12,988 fedbiomed INFO - log from: node_f290cd48-a70a-4e55-9262-81f802f9c95c / INFO - Train Epoch: 1 [2880/60000 (5%)]	Loss: 0.217385
2022-01-21 14:54:13,471 fedbiomed INFO - log from: node_f290cd48-a70a-4e55-9262-81f802f9c95c / INFO - 

In [25]:
print('Number of rounds initial             : ' , exp.rounds())
print('Number of rounds that has ben run    : ' , exp.round_current())
print('Round number for starting next round : ' , exp.round_current() + 1)
print('Round Indexes                        : ' , list(range(exp.round_current())))

Number of rounds initial             :  4
Number of rounds that has ben run    :  4
Round number for starting next round :  5
Round Indexes                        :  [0, 1, 2, 3]


In [ ]:
## New Usage exp.training_replies()

rounds = exp.current_round()

print("\nList the training rounds : ", exp.training_replies().keys())
print("\nList the nodes for the last training round and their timings : ")
for r in exp.training_replies().keys():
    round_data = exp.training_replies()[r].data
    print('\n\t Round %s' % str(r+1))
    for c in range(len(round_data)):
        print("\t\t- {id} :\
        \n\t\t\trtime_training={rtraining:.2f} seconds\
        \n\t\t\tptime_training={ptraining:.2f} seconds\
        \n\t\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
            rtraining = round_data[c]['timing']['rtime_training'],
            ptraining = round_data[c]['timing']['ptime_training'],
            rtotal = round_data[c]['timing']['rtime_total']))
print('\n')

### Run Same Experiment with Multple Rounds

In [ ]:
exp.run(rounds=2)

### Changing Experiment Parameters with Setters after all The Argument is Already Set
If the `Job` is already initialize and the arguments related to model is modified, `Job` should reinitialize with the method `.set_job()`. This information is also given by Experiment after setting model file.  
  
    
    
<div class="note">
    <p>After runing the experiment changing the model might have some consequances.</p>
</div>

In [ ]:
exp.set_model_path(model_file)
exp.set_model_class('MyTrainingPlan')

In [ ]:
exp2.set_job()

#### Changing Aggregator

Aggregator should be instance of `fedbiomed.researcher.aggregators.aggregator.Aggregator`. Otherwise `set_aggregator` will raise an Expection. Aggregator should be passed as `Callable` class or alredy built object.

Following cell will raise an Exception:

In [ ]:
exp.set_aggregator('ThisIsNotAnAggregator')

Correct usage: 

In [ ]:
from fedbiomed.researcher.aggregators.fedavg import FedAverage
# Can be passed as Callable class
exp2.set_aggregator(FedAverage)

# Can be passed as already build class
fedavg = FedAverage()
exp2.set_aggregator(fedavg)

Federated parameters for each round are available in `exp.aggregated_params` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params.keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params[rounds - 1]['params'].keys())


Feel free to run other sample notebooks or try your own models :D